In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from astropy.io import fits
from scipy.interpolate import interp1d # 线性插值
import matplotlib.pyplot as plt
from scipy.signal import medfilt # 中值滤波
from sklearn.preprocessing import StandardScaler  # 数据进行标准化
import json 
import shutil

# 函数定义

## 工具函数：

### 查看一个fit文件下有什么字段

In [2]:
def list_fits_headers(fits_path):
    """
    列出压缩FITS文件中头文件的所有字段。
    
    参数：
    fits_path (str): FITS文件的路径（支持.gz压缩文件）。
    
    返回：
    headers (dict): FITS文件中头文件的所有字段及其值。
    """
    if not os.path.exists(fits_path):
        raise FileNotFoundError(f"The file {fits_path} does not exist.")
    
    headers = {}
    
    with fits.open(fits_path) as hdul:
        for idx, hdu in enumerate(hdul):
            header = hdu.header
            headers[f"HDU_{idx}"] = dict(header.items())
    
    return headers

# # 示例用法
# fits_path = r'G:\Star\1_Data_download_and_preprocessing\Denoising_reference_set\download\FITSDATA\high_0_10\spec-55863-B6301_sp03-211.fits.gz'
# headers = list_fits_headers(fits_path)
# for hdu_name, header in headers.items():
#     print(f"HDU Name: {hdu_name}")
#     for key, value in header.items():
#         print(f"  {key}: {value}")
#     print()

### 遍历目录all_path：

功能描述

    函数遍历指定目录（包括其子目录）下的所有文件，并返回一个包含所有文件名的列表。

参数

    dirname (str): 需要遍历的目录路径字符串。
返回值

    返回一个列表，包含遍历得到的所有文件的名称（不包括路径）。

In [3]:
def all_path(dirname):
    files_names = []
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            files_names.append(filename)
    return files_names

## 1.Wavelength Analysis 波长分析(wave_analyse)

wave_analyse 函数用于分析给定目录下的 FITS 文件中记录的天体光谱数据，计算并返回给定数据集中天体光谱的蓝端和红端波长范围的最小和最大值，同时考虑了红移的影响。

参数

    DATASET_PATH (str): 指向包含光谱数据概览的 CSV 文件的路径。该文件用于辅助分析，但本函数示例中未直接使用该参数。
    FITS_PATH (str): 包含 FITS 文件的目录路径。这些 FITS 文件包含了天体光谱数据。
返回值:四个浮点数，分别是：

    B_fact_min: 所有考察光谱中，蓝端波长的最小值。
    B_fact_max: 所有考察光谱中，蓝端波长的最大值。
    R_fact_min: 所有考察光谱中，红端波长的最小值。
    R_fact_max: 所有考察光谱中，红端波长的最大值。

In [4]:
# 定义一个函数，用于分析波长数据
def wave_analyse(DATASET_PATH, FITS_PATH):
    # 蓝端波长范围的最小和最大值
    B_wave_min = np.log10(3800)
    B_wave_max = np.log10(5700)

    # 红端波长范围的最小和最大值
    R_wave_min = np.log10(5900)
    R_wave_max = np.log10(8800)
    
    # 初始化存储蓝端和红端波长数据的列表
    B_wave = []
    R_wave = []
    label = []
    
    # 读取数据集路径下的CSV文件
    data = pd.read_csv(DATASET_PATH)  
    
    # 获取所有FITS文件的路径
    file_names = all_path(FITS_PATH)
    print(len(file_names))
    file_names = np.array(file_names)
    
    # 遍历每个FITS文件
    for i in tqdm(range(len(file_names))):
        try:  
            # 打开FITS文件
            f = fits.open(FITS_PATH + file_names[i])
            try:
                # 尝试获取第一个HDU的流量和波长数据
                F = f[0].data[0]  # 流量
                W = f[0].data[2]  # 波长
            except:
                # 如果第一个HDU中没有数据，则尝试从第二个HDU中获取
                F = f[1].data["FLUX"][0]
                W = f[1].data["WAVELENGTH"][0]
                
            # 获取红移值，并调整波长数据
            z = f[0].header['Z']  # 红移
            W = np.log10(W) - np.log10(1 + np.float64(z))
            # 根据蓝端和红端的波长范围，获取相应的波长数据
            B_W = (W > B_wave_min)*(W < B_wave_max)
            R_W = (W > R_wave_min)*(W < R_wave_max)
            # 添加蓝端波长数据
            B_wave.append(W[B_W])
            # 添加红端波长数据
            R_wave.append(W[R_W])
        except:
            # 处理异常情况，跳过当前循环
            print("Handling exceptions, skipped, indexed as : ", i)
            continue
    
    # 获取所有蓝端波长数据的最大和最小值
    B_fact_max = np.max(B_wave[0])
    B_fact_min = np.min(B_wave[0])
    for W in B_wave:
        try:
            B_fact_max = min(B_fact_max, np.max(W))
            B_fact_min = max(B_fact_min, np.min(W))
        except:
            continue
    
    # 获取所有红端波长数据的最大和最小值
    R_fact_max = np.max(R_wave[0])
    R_fact_min = np.min(R_wave[0])
    for W in R_wave:
        try:
            R_fact_max = min(R_fact_max, np.max(W))
            R_fact_min = max(R_fact_min, np.min(W))
        except:
            continue
    

    B_wave_scope = [B_fact_min, B_fact_max]
    R_wave_scope = [R_fact_min, R_fact_max]
    # 返回蓝端和红端波长的最小和最大值
    return B_wave_scope, R_wave_scope

# # 测试
# B_wave_scope, R_wave_scope = wave_analyse(DATASET_PATH, FITS_PATH)
# print(B_wave_scope, R_wave_scope)

In [5]:
# 定义一个函数，用于分析波长数据（去除了基于红移的波长矫正）
def wave_analyse(DATASET_PATH, FITS_PATH):
    # 蓝端波长范围的最小和最大值
    B_wave_min = np.log10(3800)
    B_wave_max = np.log10(5700)

    # 红端波长范围的最小和最大值
    R_wave_min = np.log10(5900)
    R_wave_max = np.log10(8800)
    
    # 初始化存储蓝端和红端波长数据的列表
    B_wave = []
    R_wave = []
    label = []
    
    # # 读取数据集路径下的CSV文件
    # data = pd.read_csv(DATASET_PATH)  
    
    # 获取所有FITS文件的路径
    file_names = all_path(FITS_PATH)
    print(len(file_names))
    file_names = np.array(file_names)
    
    # 遍历每个FITS文件
    for i in tqdm(range(len(file_names))):
        try:  
            # 打开FITS文件
            f = fits.open(FITS_PATH + file_names[i])
            try:
                # 尝试获取第一个HDU的流量和波长数据
                F = f[0].data[0]  # 流量
                W = f[0].data[2]  # 波长
            except:
                # 如果第一个HDU中没有数据，则尝试从第二个HDU中获取
                F = f[1].data["FLUX"][0]
                W = f[1].data["WAVELENGTH"][0]
                
            # 获取红移值，并调整波长数据
            # z = f[0].header['Z']  # 红移
            # W = np.log10(W) - np.log10(1 + np.float64(z))
            W = np.log10(W)
            
            
            # 根据蓝端和红端的波长范围，获取相应的波长数据
            B_W = (W > B_wave_min)*(W < B_wave_max)
            R_W = (W > R_wave_min)*(W < R_wave_max)
            # 添加蓝端波长数据
            B_wave.append(W[B_W])
            # 添加红端波长数据
            R_wave.append(W[R_W])
        except:
            # 处理异常情况，跳过当前循环
            print("Handling exceptions, skipped, indexed as : ", i)
            continue
    
    # 获取所有蓝端波长数据的最大和最小值
    B_fact_max = np.max(B_wave[0])
    B_fact_min = np.min(B_wave[0])
    for W in B_wave:
        try:
            B_fact_max = min(B_fact_max, np.max(W))
            B_fact_min = max(B_fact_min, np.min(W))
        except:
            continue
    
    # 获取所有红端波长数据的最大和最小值
    R_fact_max = np.max(R_wave[0])
    R_fact_min = np.min(R_wave[0])
    for W in R_wave:
        try:
            R_fact_max = min(R_fact_max, np.max(W))
            R_fact_min = max(R_fact_min, np.min(W))
        except:
            continue
    

    B_wave_scope = [B_fact_min, B_fact_max]
    R_wave_scope = [R_fact_min, R_fact_max]
    # 返回蓝端和红端波长的最小和最大值
    return B_wave_scope, R_wave_scope


# DATASET_PATH = r"./download/snrg_split_match_catalog/match_low_0_10.csv"
# FITS_PATH = r'./download/FITSDATA/test/'
# B_wave_scope, R_wave_scope = wave_analyse(DATASET_PATH, FITS_PATH)
# print(B_wave_scope, R_wave_scope)

## 2. 线性插值 (linear_inter)

In [6]:
# 定义一个函数进行线性插值（去除了基于红移的波长矫正）
def linear_inter(FITS_PATH, B_wave_scope, R_wave_scope,start=0,end=float('inf')):
                               
    # 计算蓝端和红端波长范围的对数值
    B_wave_min = np.log10(3800)
    B_wave_max = np.log10(5700)

    R_wave_min = np.log10(5900)
    R_wave_max = np.log10(8800)
    
    
    # 初始化存储蓝红波段流量数据的列表
    BR_flux_data = []
    # 初始化文件名列表
    file_names = []
    
    # 获取FITS文件的路径
    file_names = all_path(FITS_PATH)
    
    # 初始化错误列表
    error_list = []
    
    #定义遍历结束位置
    end = min(len(file_names), end)
    
    # 遍历文件名列表
    for i in tqdm(range(start,end)):
        try:
            # 打开FITS文件
            f = fits.open(FITS_PATH + file_names[i])
            
            try:
                # 尝试从第一个HDU获取流量和波长数据
                F = f[0].data[0] 
                W = f[0].data[2] 
            except:
                # 如果第一个HDU中没有数据，则从第二个HDU获取
                F = f[1].data["FLUX"][0]
                W = f[1].data["WAVELENGTH"][0]
            
            # # 获取红移值并调整波长数据
            # z = f[0].header['Z']  
            # W = np.log10(W) - np.log10(1 + np.float64(z))
            W = np.log10(W)
        
            # # 定义蓝端和红端波长范围
            # B_wave_scope = [B_fact_min, B_fact_max]
            # R_wave_scope = [R_fact_min, R_fact_max]
        
            # 获取蓝端的流量和波长数据
            B_flux = F[(W > B_wave_min)*(W < B_wave_max)]
            B_wave = W[(W > B_wave_min)*(W < B_wave_max)]
        
            # # 对全部数据进行线性插值
            # f = interp1d(W, F, kind = 'linear')
        
            # 获取红端的流量和波长数据
            R_flux = F[(W > R_wave_min)*(W < R_wave_max)]
            R_wave = W[(W > R_wave_min)*(W < R_wave_max)]
              
            # 分别对蓝端和红端的数据进行线性插值
            B_f = interp1d(B_wave, B_flux, kind = 'linear')
            R_f = interp1d(R_wave, R_flux, kind = 'linear')
        
            # 设置采样率为0.0001，并生成固定波长范围的蓝端和红端波长数组
            B_wave_fixed = np.arange(B_wave_scope[0], B_wave_scope[1], 0.0001)
            R_wave_fixed = np.arange(R_wave_scope[0], R_wave_scope[1], 0.0001)
        
            # 将插值后的蓝端和红端流量数据合并，并添加到列表中
            BR_flux_data.append(np.concatenate((B_f(B_wave_fixed), R_f(R_wave_fixed))))
        
        except:
            # 处理异常情况，跳过当前循环
            print("Handling exceptions, skipped, indexed as : ", i) 
            error_list.append(i)
            continue
    
    # 打印处理后的数据量和数据维度信息
    print("The magnitude of the traffic data on the common wavelength position obtained \
          after interpolation of the red and blue bands separately：", np.shape(BR_flux_data)[1])
    print("Number of processed data：", np.shape(BR_flux_data)[0])
        
    # 返回插值后的流量数据以及蓝端和红端的固定波长数组
    return BR_flux_data, B_wave_fixed, R_wave_fixed, error_list


## 3. 连续谱拟合和归一化处理 (normalization_by_polyfit)

### csp_polyfit：  多项式拟合

csp_polyfit：归一化光谱和波长，拟合出一个多项式

参数说明：

    sp: 光流量数据，一维数组。
    angs: 角度数据，一维数组，应与光流量数据在长度上匹配。
    param：
        poly_global_order: 多项式拟合的全局阶数。
        poly_upperlimit和poly_lowerlimit: 用于数据点选择的标准差倍数阈值。
        poly_del_filled: 指示如何处理不符合条件的数据点，1 表示删除，其它值表示用拟合值填充。
    
返回值：

    param：
        poly_sp_c、poly_sp_s、poly_angs_c和poly_angs_s: 分别保存流量和波长的平均值、标准差
        poly_P_g: 拟合过程中计算的多项式系数。
        poly_sp_filtered和poly_angs_filtered: 过滤后的光流量和角度数据。

In [7]:
def csp_polyfit(sp, angs, param):
    # 标准化光流量
    sp_c = np.mean(sp)   # 计算光流量的平均值，即中位流量
    sp = sp - sp_c           # 以平均值为中心调整光流量，实现光流量的中心化
    sp_s = np.std(sp)     # 计算光流量的标准差，即流量均方差
    sp = sp / sp_s        # 用标准差标准化光流量
    
    # 标准化角度
    angs_c = np.mean(angs)  # 计算波长的平均值
    angs = angs - angs_c    # 以平均值为中心调整角度
    angs_s = np.std(angs)   # 计算波长的标准差
    angs = angs / angs_s    # 用标准差标准化波长
    
    # 更新参数字典
    param['poly_sp_c'] = sp_c
    param['poly_sp_s'] = sp_s
    param['poly_angs_c'] = angs_c
    param['poly_angs_s'] = angs_s
    
    # 初始化数据标志
    data_flag = np.full(sp.shape, 1)
    
    i = 0
    con = True
    while(con):
        # 使用多项式拟合并获取系数
        P_g = np.polyfit(angs, sp, param['poly_global_order'])  # 计算多项式系数
        param['poly_P_g'] = P_g
        fitval_1 = np.polyval(P_g, angs)   # 计算多项式的拟合值
        dev = fitval_1 - sp  # 计算拟合值与实际值的偏差
        sig_g = np.std(dev)  # 计算偏差的标准差
        
        # 更新数据标志，用于识别需要保留的数据点
        data_flag_new = (dev > (-param['poly_upperlimit'] * sig_g)) * (dev < (param['poly_lowerlimit'] * sig_g))
    
        if sum(abs(data_flag_new - data_flag)) > 0:
            if param['poly_del_filled'] == 1: 
                data_flag = data_flag_new  # 更新数据标志
            else:
                fill_flag = data_flag - data_flag_new  # 计算需要填充的数据标志
                index_1 = np.where(fill_flag != 0)  # 找出需要填充的数据点
                sp[index_1] = fitval_1[index_1]  # 用拟合值填充这些数据点
        else:
            con = False  # 如果数据标志没有变化，则停止迭代
        i += 1
    #     print("iteration is %d" %i)  # 打印迭代次数，此行为调试用，通常被注释
    
    # 根据最终的数据标志筛选数据
    index_2 = np.where(data_flag != 0)
    param['poly_sp_filtered'] = sp[index_2]  # 更新参数字典，保存过滤后的光流量数据
    param['poly_angs_filtered'] = angs[index_2]  # 更新参数字典，保存过滤后的角度数据
    
    return param  # 返回更新后的参数字典


### sp_median_polyfit1stage： 连续谱拟合

对一条光谱进行一系列处理最终获得一个拟合连续谱。具体操作包括：1.用中值滤波结果预剔除异常数据点 2.拟合一个归一化后的多项式 3.反归一化得到最终拟合连续谱

使用说明

    sp_median_polyfit1stage(flux, lambda_log, param)
参数

    flux: 光通量数据，一维数组。
    lambda_log: 对数波长数据，与光通量数据相对应的一维数组。
    param: 参数字典，包含处理过程中需要的各种参数。
参数字典param的关键字段

    median_radius: 中值滤波的窗口半径。
    poly_lowerlimit: 多项式拟合预剔除下限，乘以标准差作为阈值。
    poly_upperlimit: 多项式拟合预剔除上限，乘以标准差作为阈值。
    poly_del_filled: 处理标记为填充的数据点的方法（1表示删除这些点，2表示用中值滤波结果填充）。
    poly_angs_c, poly_angs_s: 波长预处理中的中心化和缩放参数。
    poly_P_g: 连续谱拟合的多项式系数。
    poly_sp_s, poly_sp_c: 连续谱拟合后的缩放和偏移参数。
    poly_SM: 是否使用原始波长（1）或对数波长（其它值）。
返回值

    continum_fitted: 拟合的连续谱，一维数组，与输入的光通量数据对应。

In [8]:
def sp_median_polyfit1stage(flux, lambda_log,param):
    flux1 = flux  # 复制光通量数据
    lambda1 = lambda_log  # 复制对数波长数据

    # 中值滤波
    flux_median1 = medfilt(flux1, param['median_radius'])  # 使用中值滤波处理光通量数据

    # 利用中值滤波结果进行预剔除
    dev1 = flux_median1 - flux1  # 计算中值滤波后的光通量与原光通量的差值
    sigma = np.std(dev1)  # 计算差值的标准差
    # 生成数据标记，用于识别符合预剔除条件的数据点
    data_flag1 = (dev1 < (param['poly_lowerlimit'] * sigma)) * (dev1 > (-param['poly_upperlimit'] * sigma))
    
    # 利用中值滤波结果进行伪光谱线的预剔除
    fill_flag1 = 1 - data_flag1  # 生成填充标记，用于识别需要被填充的数据点
    
    # 根据参数决定如何处理填充标记指示的数据点
    if param['poly_del_filled'] == 1:
        index_1 = np.where(data_flag1)  # 找到符合条件的数据点索引
        flux1 = flux1[index_1]  # 仅保留这些数据点的光通量数据
        lambda1 = lambda1[index_1]  # 仅保留这些数据点的波长数据
    elif param['poly_del_filled'] == 2:
        index_2 = np.where(fill_flag1)  # 找到需要填充的数据点索引
        flux1[index_2] = flux_median1[index_2]  # 用中值滤波结果填充这些数据点的光通量
        
    # 迭代拟合连续谱
    param = csp_polyfit(flux1, lambda1, param)  # 执行连续谱的多项式拟合
    
    # 波长预处理
    angs = lambda1 - param['poly_angs_c']  # 波长中心化
    angs = angs / param['poly_angs_s']  # 波长缩放
    
    # 连续谱样本
    fitval_g = np.polyval(param['poly_P_g'], angs)  # 使用多项式系数计算连续谱的拟合值
    continum_fitted = fitval_g * param['poly_sp_s'] + param['poly_sp_c']  # 根据参数调整连续谱的拟合值
    
    # 根据参数选择使用原始波长或对数波长
    if param['poly_SM'] == 1: 
        angss = lambda1
    else: 
        angss = 10 ** lambda1  # 如果需要，将波长转换为对数形式
 
    return continum_fitted  # 返回拟合的连续谱

### normalization_by_polyfit： 使用拟合连续谱对原始光谱数据归一化

概述
对每条光谱的蓝端和红端分别进行连续谱拟合和归一化。先分别处理蓝端和红端的光谱，对每一部分分别进行连续谱拟合，最后利用拟合结果对原始光谱数据进行归一化处理。

参数说明

    BR_flux_data: 原始光谱数据数组，假设为二维数组，其中每一行代表一条光谱数据。
    B_wave_fixed: 蓝端波长的固定采样数组，一维数组。
    R_wave_fixed: 红端波长的固定采样数组，一维数组。
    param: (可选) 拟合参数字典，用于控制连续谱拟合的具体行为。如果未提供，则使用默认参数。
    SAVE_PATH: (可选) 字符串，表示保存处理后数据的文件路径。如果提供此参数，函数将处理后的数据保存为NumPy文件。
返回值

    BR_flux_data_fitted: 归一化后的光谱数据数组，与原始光谱数据具有相同的形状。

In [9]:
def normalization_by_polyfit(BR_flux_data, B_wave_fixed, R_wave_fixed, param=None, SAVE_PATH=None):
    """                      
    对光谱数据进行连续谱拟合和归一化处理。

    参数:
    - BR_flux_data: 原始光谱数据数组。
    - B_wave_fixed: 蓝端波长的固定采样数组。
    - R_wave_fixed: 红端波长的固定采样数组。
    - param: 拟合参数字典，可选。
    - SAVE_PATH: 保存处理后数据的文件路径，可选。
    """
    # 检查是否提供了拟合参数，如果没有，则设置默认参数
    if param is None:
        param = {'poly_global_order': 5, 'nor': 1, 'poly_lowerlimit': 3, 'poly_upperlimit': 4, 
                 'median_radius': 3, 'poly_SM': 0, 'poly_del_filled': 2}

    # 初始化一个与原始光谱数据相同形状的数组，用于存储拟合后的数据
    BR_flux_data_fitted = np.zeros_like(BR_flux_data)
    
    # 遍历每一行光谱数据
    for i in tqdm(range(BR_flux_data_fitted.shape[0])):
        
        try:
            # 对蓝端和红端波长的光谱数据分别进行连续谱拟合
            B_continum_fitted_train = sp_median_polyfit1stage(BR_flux_data[i][:len(B_wave_fixed)], np.log10(B_wave_fixed), param)
            R_continum_fitted_train = sp_median_polyfit1stage(BR_flux_data[i][len(B_wave_fixed):], np.log10(R_wave_fixed), param)

            # 使用拟合得到的连续谱对原始光谱数据进行归一化
            BR_flux_data_fitted[i, :len(B_wave_fixed)] = BR_flux_data[i][:len(B_wave_fixed)] / B_continum_fitted_train
            BR_flux_data_fitted[i, len(B_wave_fixed):] = BR_flux_data[i][len(B_wave_fixed):] / R_continum_fitted_train
        except:
            print(f"Handling exceptions, indexed as: {i}")
            continue

    # 如果提供了保存路径，将处理后的数据保存到指定文件
    if SAVE_PATH:
        np.save(SAVE_PATH, BR_flux_data_fitted)
    
    return BR_flux_data_fitted

# 注意: sp_median_polyfit1stage 需要根据实际的函数定义和参数进行调整。

## 4. 用3σ原则处理异常值和标准化处理

### three_sigma：异常值处理子函数

函数基于3σ原则（三西格玛规则）对光谱数据中的异常值进行识别和处理。先假设数据遵循正态分布，数据集中在均值的±3σ（标准差）区间内的概率为99.73%，因此位于这个区间之外的点被视为异常值。该函数计算给定光谱数据片段的平均值和标准差，识别异常值，并将这些异常值替换为平均值，最后返回处理后的光谱数据点列表。

In [10]:
def three_sigma(Flux, flux_index, sigma_rate=3):
    mean_value = Flux[flux_index].mean()  # 计算指定索引光谱数据的平均值
    std_value = Flux[flux_index].std()  # 计算指定索引光谱数据的标准差
    # 根据3σ原则确定异常值规则：异常值为超出平均值±3倍标准差的数据点
    rule = (mean_value - sigma_rate * std_value > Flux[flux_index]) | (mean_value + sigma_rate * std_value < Flux[flux_index])
    indice = np.arange(Flux[flux_index].shape[0])[rule]  # 获取所有异常值的索引位置
    Flux_points = []  # 初始化一个列表，用于存放处理后的光谱数据点
    for i in range(Flux[flux_index].shape[0]):  # 遍历光谱数据点
        if i in indice:  # 如果当前点是异常值
            Flux_points.append(mean_value)  # 用平均值替换异常值
        else:
            Flux_points.append(Flux[flux_index][i])  # 正常值保持不变
    return Flux_points  # 返回处理后的光谱数据点列表


### three_sigma_processing：

用于批量处理光谱数据集，应用3σ原则去除异常值并进行后续的标准化处理。

In [11]:
def three_sigma_processing(Flux, SAVE_PATH_3sigma=None,failed_indices=[]):
    Flux_3sigma = []  # 初始化一个列表，用于存储所有光谱数据的3σ处理结果
    for i in tqdm(range(Flux.shape[0])):  # 遍历所有光谱数据
        Flux_3sigma.append(three_sigma(Flux, i, sigma_rate=3))  # 对每条光谱数据应用3σ处理
    Flux_3sigma = np.array(Flux_3sigma)  # 将处理结果列表转换为NumPy数组
    
    if np.isinf(Flux_3sigma).any():
        print("数据中存在无穷大值。")
        # 将无穷大值替换为 NaN
        Flux_3sigma[np.isinf(Flux_3sigma)] = np.nan

    rows_with_nan = np.isnan(Flux_3sigma).any(axis=1)
    indices_of_rows_with_nan = np.where(rows_with_nan)[0]
    
    if indices_of_rows_with_nan.size > 0:
        
        print("indices_of_rows_with_nan:",indices_of_rows_with_nan)
        Flux_3sigma = np.delete(Flux_3sigma, indices_of_rows_with_nan, axis=0)
        failed_indices = get_original_indices(failed_indices, indices_of_rows_with_nan)
    

    sc = StandardScaler()  # 创建一个StandardScaler对象，用于后续的标准化处理
    Flux_3sigma_sc_T = sc.fit_transform(Flux_3sigma.T)  # 对3σ处理后的光谱数据进行标准化
    Flux_3sigma_sc = Flux_3sigma_sc_T.T  # 转置回原来的形状
    
    
    
    if SAVE_PATH_3sigma:
        np.save(SAVE_PATH_3sigma, Flux_3sigma_sc)  # 将标准化后的光谱数据保存到指定路径
        
    return  Flux_3sigma_sc, failed_indices

# Flux = np.array([
#         [1, 2, 3, 4,123,4848,12,45],
#         [5, np.inf, 7, 8,123,4848,12,45],  # 包含无穷大
#         [9, 10, np.nan, 12,123,4848,12,45],  # 包含NaN
#         [13, 14, 15, 16,123,4848,12,45],
#         [13, 14, 15, 16,123,4848,12,45],
#         [13, 14, 15, 16,123,4848,12,45],
#         [9, 10, np.nan, 12,123,4848,12,45],
#     ])
# failed_indices = []  # 假设在原始数据中出错的索引
# Flux_3sigma_sc, failed_indices = three_sigma_processing(Flux, failed_indices=failed_indices)
# failed_indices

In [12]:
def get_original_indices(failed_indices, new_failed_index):
    # failed_indices: list of indices of previous failed data in original data
    # new_failed_index: list of indices of newly failed data in the filtered data
    
    # Create a mapping from new data index to original data index
    failed_indices_raw = failed_indices[:]
    for i in new_failed_index:
        for j in range(len(failed_indices)):
            if failed_indices[j] <= i:
                i += 1
            else:
                failed_indices_raw.append(i)
                break
        else:
            failed_indices_raw.append(i)
    failed_indices_raw.sort()        
    return failed_indices_raw        

## 5.提取info文件

In [13]:
def extract_fits_info(FITS_PATH, start=0, end = float('inf'), failed_indice=[],save_path=None):
    """
    提取指定范围内的FITS文件中的OBSID、SNRG、RA、DEC字段，并存储到一个DataFrame中。
    
    参数：
    FITS_PATH (str): FITS文件的目录路径。
    start (int): 起始索引（包含）。
    end (int): 结束索引（不包含）。
    failed_indice (list): 要跳过的文件索引列表。
    
    返回：
    df (pd.DataFrame): 包含提取信息的DataFrame，列名为obsid、snrg、ra、dec。
    """
    data = []
    
    # 获取FITS文件的路径
    file_names = all_path(FITS_PATH)
    
    #定义遍历结束位置
    end = min(len(file_names), end)
    
    if failed_indice:
        # 如果不为空，将所有元素加上start
        failed_indice = [x + start for x in failed_indice]
    
    for i in tqdm(range(start,end)):
        
        # 检查当前索引是否在failed_indice中
        if i in failed_indice:
            continue
        
        fits_path = os.path.join(FITS_PATH, file_names[i])
        with fits.open(fits_path) as hdul:
            header = hdul[0].header
            obsid = header.get('OBSID', None)
            snrg = header.get('SNRG', None)
            ra = header.get('RA', None)
            dec = header.get('DEC', None)
            data.append([obsid, snrg, ra, dec])
    
    df = pd.DataFrame(data, columns=['obsid', 'snrg', 'ra', 'dec'])
    
        # 如果提供了保存路径，则保存DataFrame到CSV文件
    if save_path:
        df.to_csv(save_path, index=False)
        
    return df

# FITS_PATH = r"G:\Star\1_Data_download_and_preprocessing\Denoising_reference_set\download\FITSDATA\test/"
# failed_indice =[1, 2, 3, 4, 6, 9]
# end = 100
# info_df = extract_fits_info(FITS_PATH,start=50, end=end, failed_indice=failed_indice)
# info_df.shape

# 预处理全流程打包：

In [14]:
def process_large_fits_files(FITS_PATH, SAVE_PATH = None, n=100000, wave_scope_path="./wave_scope.json",start=0):
    """
    处理大量FITS文件的函数，执行数据插值、连续谱拟合、3σ处理和信息提取等操作。
    
    参数：
    FITS_PATH (str): FITS文件的目录路径。
    SAVE_PATH (str): 保存处理后数据的路径。
    info_save_path (str): 保存提取信息的CSV文件路径。
    n (int): 每次处理的最大文件数量。
    wave_scope_path (str): 波长范围的JSON文件路径。
    """
    
    # 1.读取最小和最大值。保证和训练集处理流程一致
    with open(wave_scope_path, "r") as file:
        wave_scope = json.load(file)
    B_wave_scope = wave_scope['B_wave_scope']
    R_wave_scope = wave_scope['R_wave_scope']
    
    # 创建保存目录
    flux_save_dir = os.path.join(SAVE_PATH, 'flux')
    info_save_dir = os.path.join(SAVE_PATH, 'info')
    os.makedirs(flux_save_dir, exist_ok=True)
    os.makedirs(info_save_dir, exist_ok=True)
    num_files = len(all_path(FITS_PATH)) // n + 1
    
    # 将一个大文件分多个部分处理
    for i in range(start,num_files):
        start = i * n
        end = (i + 1) * n
        
         # 定义保存路径
        flux_save_path = os.path.join(flux_save_dir, "{}_flux.npy".format(i))
        info_save_path = os.path.join(info_save_dir, "{}_info.csv".format(i))
        print(flux_save_path)
        
        # 2.在范围内进行插值，统一波长范围和数据格式
        BR_flux_data, B_wave_fixed, R_wave_fixed, failed_indice = linear_inter(FITS_PATH, B_wave_scope, R_wave_scope, start=start, end=end)

        # 3.进行连续谱拟合，并以此对原始光谱数据归一化
        BR_flux_data_fitted = normalization_by_polyfit(BR_flux_data, B_wave_fixed, R_wave_fixed, param=None, SAVE_PATH=None)

        # 4.基于3σ原则对光谱数据中的异常值进行识别和处理，并归一化，最后将数据保存到save_path
        Flux_3sigma_sc, failed_indice = three_sigma_processing(BR_flux_data_fitted, SAVE_PATH_3sigma=flux_save_path,failed_indices=failed_indice)
        print("failed_indice:", failed_indice)
        # 5. 提取当前范围内的FITS文件信息（OBSID、SNRG、RA、DEC），并将信息保存到CSV文件中
        info_df = extract_fits_info(FITS_PATH, start, end, failed_indice=failed_indice, save_path=info_save_path)
        print("flux shape:",Flux_3sigma_sc.shape)
        print("info shape:",info_df.shape)

        
    return Flux_3sigma_sc, info_df

## 处理 16-20的数据

In [15]:
FITS_PATH = r'../catalog_download/FITS/16-20/'
SAVE_PATH = r'../Fits_preprocessed/16-20/'

n = 100000   

process_large_fits_files(FITS_PATH, SAVE_PATH = SAVE_PATH , n=n)

../Fits_preprocessed/16-20/flux\0_flux.npy


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [09:49<00:00, 169.58it/s]


The magnitude of the traffic data on the common wavelength position obtained           after interpolation of the red and blue bands separately： 3450
Number of processed data： 100000


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [13:25<00:00, 124.12it/s]


failed_indice: []


...

 82%|█████████████████████████████████████████████████████████████▋             | 17167/20858 [01:26<00:17, 211.23it/s]C:\Users\ZhaoXR\AppData\Local\Temp\ipykernel_9556\1774348333.py:30: RuntimeWarning: invalid value encountered in divide
  BR_flux_data_fitted[i, len(B_wave_fixed):] = BR_flux_data[i][len(B_wave_fixed):] / R_continum_fitted_train
100%|███████████████████████████████████████████████████████████████████████████| 20858/20858 [02:35<00:00, 134.41it/s]


indices_of_rows_with_nan: [17186]
failed_indice: [17186]


100%|███████████████████████████████████████████████████████████████████████████| 20858/20858 [00:28<00:00, 742.83it/s]


flux shape: (20857, 3450)
info shape: (20857, 4)


(array([[-3.74697278, -3.78235195, -3.69388501, ...,  0.88686407,
          0.59490921,  0.05755014],
        [-3.95207231, -0.09017275, -2.72029037, ..., -1.65022614,
         -1.54198895, -2.50544793],
        [-0.10987553, -4.01185717, -1.89651461, ...,  1.94595702,
          0.17206218,  0.82726523],
        ...,
        [-0.03144827, -0.03144827, -0.03144827, ..., -1.00664445,
         -1.01485043, -0.92718756],
        [-0.03503371, -2.8985549 , -0.22527337, ..., -0.03503371,
         -0.03503371, -0.03503371],
        [-2.69544571, -2.15008595,  0.96185225, ...,  0.78169599,
          0.75834781,  0.84654214]]),
             obsid   snrg          ra        dec
 0      1058111173  18.11   96.159323  54.537961
 1      1058111217  17.66   97.274598  54.718099
 2      1058112005  16.79   99.564636  53.826233
 3      1058112010  18.28   99.751652  54.073488
 4      1058112019  16.12   99.457830  53.805246
 ...           ...    ...         ...        ...
 20852  1128116025  16.79  281

## 处理 30-35的数据

In [16]:
FITS_PATH = r'../catalog_download/FITS/30-35/'
SAVE_PATH = r'../Fits_preprocessed/30-35/'

n = 100000   

process_large_fits_files(FITS_PATH, SAVE_PATH = SAVE_PATH , n=n)

../Fits_preprocessed/30-35/flux\0_flux.npy


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [10:04<00:00, 165.47it/s]


The magnitude of the traffic data on the common wavelength position obtained           after interpolation of the red and blue bands separately： 3450
Number of processed data： 100000


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [12:47<00:00, 130.32it/s]


failed_indice: []


...

 30%|███████████████████████                                                       | 981/3312 [00:05<00:12, 179.85it/s]C:\Users\ZhaoXR\AppData\Local\Temp\ipykernel_9556\1774348333.py:30: RuntimeWarning: invalid value encountered in divide
  BR_flux_data_fitted[i, len(B_wave_fixed):] = BR_flux_data[i][len(B_wave_fixed):] / R_continum_fitted_train
100%|█████████████████████████████████████████████████████████████████████████████| 3312/3312 [00:24<00:00, 135.74it/s]


indices_of_rows_with_nan: [984]
failed_indice: [984]


100%|█████████████████████████████████████████████████████████████████████████████| 3312/3312 [00:04<00:00, 779.32it/s]

flux shape: (3311, 3450)
info shape: (3311, 4)


(array([[-0.05129747, -2.5004574 , -2.38433456, ...,  0.05830685,
         -0.27884234, -0.27488583],
        [-0.17716117, -3.33416107, -3.39773674, ...,  0.20671677,
          0.41901137,  0.16856611],
        [-0.12313798, -3.9372145 , -0.12313798, ...,  0.1459633 ,
          0.18669405,  0.11261704],
        ...,
        [-0.15333518, -0.15333518, -0.91734887, ...,  2.02395608,
          2.04970277,  0.91095282],
        [-0.13486798, -0.68260919, -1.03729732, ..., -0.66805934,
         -1.09318194, -0.9668637 ],
        [-2.50741085, -0.06669427, -0.06669427, ..., -0.02669275,
         -0.32519711, -0.08979576]]),
            obsid   snrg          ra        dec
 0     1111015106  34.83  195.814089  11.710451
 1     1111015112  31.34  195.552356  11.839663
 2     1111015118  32.89  195.642502  11.799528
 3     1111015119  31.06  195.530596  11.772068
 4     1111015149  30.09  195.514151  12.047389
 ...          ...    ...         ...        ...
 3306  1128116056  31.47  283.267134 

## 处理 30-35的数据

In [17]:
FITS_PATH = r'../catalog_download/FITS/13-16/'
SAVE_PATH = r'../Fits_preprocessed/13-16/'

n = 100000   

process_large_fits_files(FITS_PATH, SAVE_PATH = SAVE_PATH , n=n)

../Fits_preprocessed/13-16/flux\0_flux.npy


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [10:18<00:00, 161.71it/s]


The magnitude of the traffic data on the common wavelength position obtained           after interpolation of the red and blue bands separately： 3450
Number of processed data： 100000


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [12:45<00:00, 130.62it/s]


failed_indice: []


...

100%|███████████████████████████████████████████████████████████████████████████| 64832/64832 [08:13<00:00, 131.42it/s]


failed_indice: []


100%|███████████████████████████████████████████████████████████████████████████| 64832/64832 [01:44<00:00, 621.36it/s]


flux shape: (64832, 3450)
info shape: (64832, 4)


(array([[-2.97828739,  0.88465233, -1.8623727 , ..., -0.18062994,
         -0.17502067, -0.25175281],
        [-0.00428256, -0.00428256, -0.00428256, ...,  0.38326399,
          0.316794  ,  0.00576271],
        [-3.47386224, -3.63560934, -0.09552162, ..., -2.97246436,
         -2.45297556, -2.97563499],
        ...,
        [-0.05584169, -0.05584169, -4.14034076, ..., -0.05584169,
         -0.05584169, -0.05584169],
        [ 0.69669136, -2.03940532,  0.50247878, ...,  1.42897857,
         -0.08351701,  1.37173952],
        [ 0.06477717, -1.68182388,  2.25408616, ..., -0.39503176,
         -0.69863197, -0.69099562]]),
             obsid   snrg          ra        dec
 0       892716117  15.26  223.691252  51.889493
 1       892716119  15.42  223.512896  51.799577
 2       892716156  15.33  222.652846  51.510397
 3       892716202  13.62  221.992778  51.841686
 4       892716227  13.50  222.805132  52.095758
 ...           ...    ...         ...        ...
 64827  1128115224  15.53  284